# Clash of Clans CWL
## Create Dataframes with League Big

* Your Name: Nolan Clark
* Name of API: Clash of Clans API
* Link to main page of API documentation: https://developer.clashofclans.com/#/documentation

In [1]:
# Imports go here
import requests
import pandas as pd
import numpy as np
import MyKeys
import matplotlib.pyplot as plt
import json
import datetime
import seaborn as sns
import CWL_Functions as cf
from collections import Counter
from collections import defaultdict

In [2]:
# API Token used with the documented Authorization header. 
# The headers are the necessary params of each endpoint
MyToken = MyKeys.MyToken
headers = {'authorization': 'Bearer '+(MyToken), 'Accept': 'application/json'}
base = 'https://api.clashofclans.com/v1/'

In [9]:
# open JSON file to update with current CWL
with open("PipelineRunOneVERIFIED.json") as fp:
    dictObj = json.load(fp)

In [4]:
# Used for excluding the leagues in dictionary that do not have tags
def leagueBinsNotEmpty(dictObj):
    leagueBinCounts ={key: (len(dictObj[key])) for key in  dictObj.keys()}
    fullBins = {k:v for k,v in leagueBinCounts.items() if v >0}
    leagueList = [league for league in fullBins.keys()]
    return fullBins, leagueList
    

In [51]:
def get_attacks(war_id):
    war_id_transformed = war_id.strip('#')
    url=('https://api.clashofclans.com/v1/clanwarleagues/wars/%23'+(war_id_transformed))
    response = requests.get(url, headers=headers, timeout=30)
    member = response.json()['clan'].get('members')
    o_member = response.json()['opponent'].get('members')
    
    clan_name = response.json()['clan'].get('name')
    clan_tag = response.json()['clan'].get('tag')
    opp_clan_name = response.json()['opponent'].get('name')
    opp_clan_tag = response.json()['opponent'].get('tag')
    
    data = []
    attacks = []
    defense_bin = []
    
# grab first clan Name, ID, TH, Attacks, and Defense stats
    for mem in member:
        output = {'name': mem.get('name'),
                  'tag' : mem.get('tag'),
                  'clan' : clan_name,
                  'clan_tag' : clan_tag,
                  'mapPosition': mem.get('mapPosition'),
                  'TH_lvl' : mem.get('townhallLevel')
                 }
        data.append(output)
        
        
        attack = mem.get('attacks')
        if attack == None:
            attack=[{'attackerTag': mem.get('tag'),
             'defenderTag': None,
             'stars': None,
             'destructionPercentage': None,
             'order': None,
             'duration': None}]
        attacks.append(attack[0])
        
        ['opponentAttacks']
        defense = mem.get('bestOpponentAttack')
        if defense == None:
            defense = {'defenderTag':None,
                         'stars': None,
                         'destructionPercentage': None,
                         'order': None,
                         'duration': None
                        }
        defense_bin.append(defense)
        
# grab second clan Name, ID, TH, Attacks, and Defense stats        
    for mem in o_member:
        output = {'name': mem.get('name'),
                  'tag' : mem.get('tag'),
                  'clan' : opp_clan_name,
                  'clan_tag' : opp_clan_tag,
                  'mapPosition': mem.get('mapPosition'),
                  'TH_lvl' : mem.get('townhallLevel')
                 }
        data.append(output)
        attack = mem.get('attacks')
        if attack == None:
            attack=[{'attackerTag': mem.get('tag'),
             'defenderTag': None,
             'stars': None,
             'destructionPercentage': None,
             'order': None,
             'duration': None}]
        attacks.append(attack[0])
        
        defense = mem.get('bestOpponentAttack')
        if defense == None:
            defense = {'defenderTag':None,
                         'stars': None,
                         'destructionPercentage': None,
                         'order': None,
                         'duration': None
                        }
            
        defense_bin.append(defense)
   
    
    return data,attacks,defense_bin

In [52]:
def frame_attacks(war_id, league=None):
    returned_war_attacks = get_attacks(war_id)

# set info df for merge
    info = pd.DataFrame(returned_war_attacks[0])

# set attack df for merge -- specify attack stats
    atk = pd.DataFrame(returned_war_attacks[1])
    atk.rename(columns={'attackerTag':'tag'}, inplace=True)
    atk = atk[['tag','stars','destructionPercentage','order','duration']]
    atk.rename(columns={'stars':'A_stars',
                       'destructionPercentage':'A_Percent',
                       'order':'A_order',
                       'duration':'A_duration'
                       }, inplace=True)
    
# set defense df for merge -- specify defense stats
    defs = pd.DataFrame(returned_war_attacks[2])
    defs = defs[['defenderTag','stars','destructionPercentage','order','duration']]

    defs.rename(columns={'defenderTag':'tag',
                         'stars': 'D_stars',
                         'destructionPercentage':'D_Percent',
                         'order':'D_order',
                         'duration':'D_duration'
                        }, inplace = True)



    df = pd.merge(info,atk, how = 'inner', on = 'tag')
    df_all = pd.merge(df, defs, how = 'left', on = 'tag')
    
    df_all['league'] = league
    df_all['attack_trans'] = df_all.A_stars*df_all.A_Percent
    df_all['def_trans'] = df_all.D_stars*df_all.D_Percent
    df_all['star_diff'] = (df_all.A_stars - df_all.D_stars)

    
    return df_all

In [53]:
def getClanOppTags(warTag):
    war_id_transformed = warTag.strip('#')
    url=('https://api.clashofclans.com/v1/clanwarleagues/wars/%23'+(war_id_transformed))
    response = requests.get(url, headers=headers, timeout=30)
    
    clans_in_this_war = [response.json()['clan'].get('tag'),response.json()['opponent'].get('tag')]
    
    return clans_in_this_war

In [54]:
def grab_cwl_stars(warTag):
    war_id_transformed = warTag.strip('#')
    url=('https://api.clashofclans.com/v1/clanwarleagues/wars/%23'+(war_id_transformed))
    response = requests.get(url, headers=headers, timeout=30)
    
    clan_tag = response.json()['clan'].get('tag')
    opp_tag = response.json()['opponent'].get('tag')
    
    clan_star = response.json()['clan'].get('stars')
    clan_percent = response.json()['clan'].get('destructionPercentage')
    opp_star = response.json()['opponent'].get('stars')
    opp_percent = response.json()['opponent'].get('destructionPercentage')
    
    if clan_star == opp_star:
        if clan_percent > opp_percent:
            clan_star+=10
        elif  clan_percent < opp_percent:
            opp_star+=10
    if clan_star > opp_star:
        clan_star+=10
    else:
        opp_star+=10
        
    return clan_tag, clan_star, opp_tag, opp_star, clan_percent, opp_percent


# Dataframe a CWL League for model train


clan_count_bucket = Counter()
cwl_day_bucket = pd.DataFrame(data = None)
leagues =leagueBinsNotEmpty(dictObj)

# Clan Counter to avoid double grabbing -- works, but pipeline requests info that is not included

for league in leagues[1]:
    for warTag in dictObj[league]:
        df_day = frame_attacks(warTag, league)
        clan_count_bucket.update(df_day[1])
        if not ((clan_count_bucket.get(df_day[1][0]) > 7) or (clan_count_bucket.get(df_day[1][1]) > 7)):
            cwl_day_bucket=cwl_day_bucket.append(df_day[0], ignore_index=True)

In [55]:
{key: (len(dictObj[key])) for key in  dictObj.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 28,
 'Silver League II': 0,
 'Silver League I': 140,
 'Gold League III': 168,
 'Gold League II': 840,
 'Gold League I': 1764,
 'Crystal League III': 3472,
 'Crystal League II': 5040,
 'Crystal League I': 8512,
 'Master League III': 6636,
 'Master League II': 3556,
 'Master League I': 2688,
 'Champion League III': 2100,
 'Champion League II': 1484,
 'Champion League I': 448}

In [57]:
leagues = [ 'Gold League III',
            'Gold League II',
            'Gold League I',
            'Crystal League III',
            'Crystal League II',
            'Crystal League I',
            'Master League III',
            'Master League II',
            'Master League I',
            'Champion League III',
            'Champion League II',
            'Champion League I']

In [58]:
# Dataframe a CWL League for model train -- CWL stars grab


clan_count_bucket = Counter()
cwl_day_bucket = pd.DataFrame(data = None)
#leagues =leagueBinsNotEmpty(dictObj)

# Clan Counter to avoid double grabbing -- optimized

for league in leagues:
    print(league)
    for warTag in dictObj[league]:
        roundClanTags = getClanOppTags(warTag)
        clan_count_bucket.update(roundClanTags)
        
        if not ((clan_count_bucket.get(roundClanTags[0]) > 7) or (clan_count_bucket.get(roundClanTags[1]) > 7)):

            df_day = frame_attacks(warTag, league)
            cwl_day_bucket=cwl_day_bucket.append(df_day, ignore_index=True)

Gold League III
Gold League II
Gold League I
Crystal League III
Crystal League II
Crystal League I
Master League III
Master League II
Master League I
Champion League III
Champion League II
Champion League I


In [59]:
# Collect matchups for all rounds in all leagues - group number collect


clan_count_bucket = Counter()
cwl_groups = defaultdict(list)
cwl_star_bucket = defaultdict(int)
cwl_damage_bucket = defaultdict(float)

#leagues =leagueBinsNotEmpty(dictObj)

# Clan Counter to avoid double grabbing -- optimized

for league in leagues:
    print(league)
    for warTag in dictObj[league]:
        roundClanTags = getClanOppTags(warTag)
        clan_count_bucket.update(roundClanTags)
        
        if not ((clan_count_bucket.get(roundClanTags[0]) > 7) or (clan_count_bucket.get(roundClanTags[1]) > 7)):
            # group num collect
            cwl_groups[roundClanTags[0]].append(roundClanTags[1])
            cwl_groups[roundClanTags[1]].append(roundClanTags[0])
            # cwl star collect
            rs = grab_cwl_stars(warTag)
            cwl_star_bucket[rs[0]]+=rs[1]
            cwl_star_bucket[rs[2]]+=rs[3]
            # cwl damage collect
            cwl_damage_bucket[rs[0]]+=rs[4]
            cwl_damage_bucket[rs[2]]+=rs[5]

In [60]:
# ADD key to group values before sorting and removing duplicates
for key, group in cwl_groups.items():
    group.append(key)
# list of sorted groups
group_sets =[sorted(item) for item in cwl_groups.values()]

In [61]:
# Unique list of groups in comprehension

cwl_unique_groups = [list(x) for x in set(tuple(x) for x in group_sets)]

unique_df=pd.DataFrame(cwl_unique_groups)
uni_df_T = unique_df.T

groups_for_merge = pd.melt(uni_df_T, var_name='group_num', value_name='clan_tag')

In [62]:
groups_for_merge

,group_num,clan_tag
0,0,#22QCRQQPL
1,0,#2YYC0LVG0
2,0,#80RUCPUG
3,0,#LGRQ9J0L
4,0,#LVPL8VJL
...,...,...
10323,1290,#928P89U9
10324,1290,#99UP9G0G
10325,1290,#9CC8U2VC
10326,1290,#J22PPCQ


In [63]:
stars=pd.Series(cwl_star_bucket).to_frame(name = 'Tot_Clan_Stars')

In [64]:
stars.index

Index(['#2G00GQL20', '#29PRQYJYQ', '#2LRCYYU0V', '#2QV9UQY9U', '#8YCCYQUG',
       '#2YUR8Q9RC', '#2QYCQQ92U', '#PR8PUVG0', '#2Q9JYLUQL', '#8JJG8R9R',
       ...
       '#2PRGLLP2Y', '#L02LQQV2', '#9QC0L8YG', '#22CYQ2J2', '#8J909CLU',
       '#YPGQCCP', '#UU0L2U9', '#YQC8Q8UR', '#88U2YRRG', '#2LJCYG2YJ'],
      dtype='object', length=10328)

In [65]:
clans_ds=pd.Series(cwl_damage_bucket).to_frame(name='Tot_Clan_Damage').join(stars)

In [66]:
stars_groups=groups_for_merge.set_index('clan_tag').join(clans_ds).reset_index()

In [67]:
# Rank the groups for final placement with total clan damage as tie breaker
stars_groups['group_rank'] = stars_groups.sort_values(by = 'Tot_Clan_Damage', 
                         ascending=False).groupby('group_num')['Tot_Clan_Stars'].rank(method = 'first',
                                                                                      ascending = False)

# Problem-Solution: including a total damage variable prevented 3.5 type rankings
# Validation check on group 201: Confirmed #9CGGRUUQ promo + #JUPJJV0G remained

In [62]:
# Rank the groups to have final placements
stars_groups['group_rank'] =  stars_groups.groupby('group_num')['Tot_Clan_Stars'].rank(ascending=False)

In [68]:
stars_groups[stars_groups['group_num'] == 9].sort_values(by='group_rank')

,clan_tag,group_num,Tot_Clan_Damage,Tot_Clan_Stars,group_rank
72,#2UVP922V,9,553.866667,293,1.0
76,#LY8JVLRY,9,547.800000,268,2.0
74,#GR08GR92,9,540.800000,265,3.0
79,#UU90VQJ,9,525.666667,255,4.0
78,#RLURP2GP,9,522.466667,252,5.0
75,#JPJRRVYV,9,506.933333,231,6.0
73,#98LV2U8V,9,472.000000,208,7.0
77,#P9U9CQG9,9,414.866667,177,8.0


In [70]:
def promo_results(x):
    if x == 1:
        return 'PROMOTED'
    elif x>=7:
        return 'DROPPED'
    else:
        return 'REMAINED'

In [71]:
stars_groups['result']=stars_groups['group_rank'].apply(promo_results)

In [103]:
# Write to file Clans measures
stars_groups.to_csv(r'JulyCWLgroupPerformanceDF.csv', index=True)

In [73]:
pd.unique(stars_groups.group_rank)

array([4., 7., 3., 5., 6., 2., 8., 1.])

## Compare pipelines

previous pipeline misread "NA" as pd.nan --- possibly on the pd.read_csv issue

current pipeline correctly read name as "NA"

PROBLEM SOLVED: pd.read_csv options = keep_default_na=False, na_values = '' 
writing to csv turns all NaNs into blanks, while reading csv interprets many options on default.


In [77]:
cwl_df=cwl_day_bucket.merge(stars_groups[['clan_tag','group_rank','result']], on = 'clan_tag')

In [102]:
cwl_df

,name,tag,clan,clan_tag,mapPosition,TH_lvl,A_stars,A_Percent,A_order,A_duration,D_stars,D_Percent,D_order,D_duration,league,attack_trans,def_trans,star_diff,group_rank,result
0,Killer C+,#QYP80QQLU,❤SRI LANKA❤,#2G00GQL20,7,13,2.0,97.0,13.0,170.0,3.0,100.0,8.0,122.0,Gold League III,194.0,300.0,-1.0,3.0,REMAINED
1,MINAGA MINRUTHA,#Y2RG98QL2,❤SRI LANKA❤,#2G00GQL20,12,11,NaN,NaN,NaN,NaN,3.0,100.0,7.0,72.0,Gold League III,NaN,300.0,NaN,3.0,REMAINED
2,$DARK LION$ 5,#QG0CYLQGQ,❤SRI LANKA❤,#2G00GQL20,2,13,3.0,100.0,16.0,125.0,3.0,100.0,9.0,109.0,Gold League III,300.0,300.0,0.0,3.0,REMAINED
3,Killer B+,#YUVC9G0Y2,❤SRI LANKA❤,#2G00GQL20,4,12,3.0,100.0,12.0,161.0,2.0,99.0,11.0,139.0,Gold League III,300.0,198.0,1.0,3.0,REMAINED
4,©hathu®a,#Q0JR8G8YJ,❤SRI LANKA❤,#2G00GQL20,6,13,3.0,100.0,17.0,123.0,3.0,100.0,10.0,144.0,Gold League III,300.0,300.0,0.0,3.0,REMAINED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084435,KHAN❣️THANOS,#2QCRRUQJR,MEGA Conqueror,#2LJCYG2YJ,9,15,2.0,60.0,28.0,77.0,2.0,97.0,11.0,179.0,Champion League I,120.0,194.0,0.0,7.0,DROPPED
1084436,PRADUTA #01,#VLCCCR9J,MEGA Conqueror,#2LJCYG2YJ,1,15,2.0,89.0,1.0,135.0,2.0,97.0,18.0,179.0,Champion League I,178.0,194.0,0.0,7.0,DROPPED
1084437,dreamz ❤️ jenn,#8CY2LUGJY,MEGA Conqueror,#2LJCYG2YJ,5,15,0.0,49.0,30.0,66.0,2.0,86.0,7.0,116.0,Champion League I,0.0,172.0,-2.0,7.0,DROPPED
1084438,PRADUTA #02,#8CGJQ9RCP,MEGA Conqueror,#2LJCYG2YJ,2,15,2.0,70.0,2.0,107.0,3.0,100.0,19.0,168.0,Champion League I,140.0,300.0,-1.0,7.0,DROPPED


In [80]:
cwl_df.to_csv(r'JulyModelDF.csv', index=True)

In [101]:
cwl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1084440 entries, 0 to 1084439
Data columns (total 20 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   name          1084440 non-null  object 
 1   tag           1084440 non-null  object 
 2   clan          1084440 non-null  object 
 3   clan_tag      1084440 non-null  object 
 4   mapPosition   1084440 non-null  int64  
 5   TH_lvl        1084440 non-null  int64  
 6   A_stars       949313 non-null   float64
 7   A_Percent     949313 non-null   float64
 8   A_order       949313 non-null   float64
 9   A_duration    949313 non-null   float64
 10  D_stars       932429 non-null   float64
 11  D_Percent     932429 non-null   float64
 12  D_order       932429 non-null   float64
 13  D_duration    932429 non-null   float64
 14  league        1084440 non-null  object 
 15  attack_trans  949313 non-null   float64
 16  def_trans     932429 non-null   float64
 17  star_diff     824611 non-nu